In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from tabulate import tabulate


np.set_printoptions(precision=0, suppress=True)
# Set display option to show numbers in a general number format
pd.set_option('display.float_format', '{:.0f}'.format)
pd.options.mode.copy_on_write = True
# Set display options to show all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Set display options to show full content without truncation and set a wide display width
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.width', 1000)  # Adjust the width as per your requirement

In [2]:
# read data file 
relation_df = pd.read_excel(r"C:\Users\user\OneDrive - Universiti Sains Malaysia\FYP\Final result bnlearn_df.xlsx")

In [3]:
# Define a function to find the source hierarchy
def find_source_hierarchy(target):
    source_list = []
    current_target = target

    while True:
        source = relation_df.loc[relation_df['target'] == current_target, 'source'].values
        if len(source) == 0:
            break
        source_list.append(source[0])
        current_target = source[0]

    return source_list

In [1]:
# # Create a dictionary to store target and its corresponding source hierarchy
# target_source_hierarchy = {}
# for target in relation_df['target'].unique():
#     target_source_hierarchy[target] = find_source_hierarchy(target)

# # Initialize empty lists
# target_list = []
# source_hierarchy_list = []

# # Iterate through the dictionary and append values to lists
# for target, source_hierarchy in target_source_hierarchy.items():
#     target_list.append(target)
#     source_hierarchy_list.append(source_hierarchy)
    
# # Combine the lists into a DataFrame
# combined_df = pd.DataFrame({
#     'target': target_list,
#     'source_hierarchy': source_hierarchy_list
# })

# # Convert DataFrame to HTML
# html_table = combined_df.to_html(index=False)

# # Create a scrollable div
# scrollable_div = f"""
# <div style="overflow-x: auto; white-space: nowrap;">
#     {html_table}
# </div>
# """

# # Display the scrollable div
# display(HTML(scrollable_div))

In [5]:
# read data file 
whole_df = pd.read_excel(r"C:\Users\user\OneDrive - Universiti Sains Malaysia\FYP\Dataset.xlsx")

In [7]:
# Calculate the moving average of 3 months
ma_whole_df = whole_df.rolling(window=3).mean()
# Drop the missing value
ma_whole_df = ma_whole_df.dropna()
# Reset the index
ma_whole_df = ma_whole_df.reset_index(drop=True)

In [8]:
# Grouping 'To' by 'From' and creating a DataFrame
target_df = relation_df.groupby('target')['source'].apply(list).reset_index()
target_df.columns = ['target', 'predictor']

target_df

,target,predictor
0,CA-Investments - Other_eb,[NCA-Deferred tax_eb]
1,CA-Loans to Others_eb,[NCA-Long-term investments_eb]
2,CA-Other current assets_eb,[CFO-Other current assets]
3,CA-Prepaid Expenses_eb,[NCA-Furniture and Fixtures_eb]
4,CA-Trade and other receivables_eb,[NCA-Long-term investments_eb]
5,CFI-Interest earned,[NCA-Other non-current assets_eb]
6,CFI-Other current assets,[NCA-Other non-current assets_eb]
7,CFO-Accrued liabilities,[NCA-Long-term investments_eb]
8,CFO-Advertising/Promotional,[Advertising/Promotional]
9,CFO-Cost of Labour,[Cost of Labour - COS]


In [9]:
def calculate_metrics(df1, aggregated_change):
    # Separate the columns based on prefixes
    cf_prefixes = ['CFI', 'CFO']
    bs_prefixes = ['CA', 'CL', 'NCA','SE']

    cf_cols = [col for col in df1.columns if any(col.startswith(prefix) for prefix in cf_prefixes)]
    bs_cols = [col for col in df1.columns if any(col.startswith(prefix) for prefix in bs_prefixes)]
    pl_cols = [col for col in df1.columns if col not in cf_cols and col not in bs_cols]

    # Create the separate DataFrames
    cf_df = df1[cf_cols]
    bs_df = df1[bs_cols]
    pl_df = df1[pl_cols]
    
    # Multiply values by -1 for columns not in specified variables
    cf_inflow = ['CFI-Interest earned', 'CFI-Other Miscellaneous Income', 'CFO-Trade and other receivables']

    for col in cf_df.columns:
        if col not in cf_inflow:
            cf_df[col] *= -1

    pl_income = ['Sales - wholesale', 'Service/Fee Income', 'Depreciation',
           'Exchange Gain or Loss', 'Penalties and settlements',
           'Interest earned', 'Loss on disposal of assets',
           'Other Miscellaneous Income', 'Other operating income']

    for col in pl_df.columns:
        if col not in pl_income:
            pl_df[col] *= -1


    # List comprehension to find columns starting with 'CL'
    bs_cl = [col for col in bs_df.columns if col.startswith('CL')]
    bs_cl.append('NCA-Accumulated depreciation on property, plant and equipment_eb')

    # Multiply the values in CL columns by -1
    bs_df[bs_cl] *= -1
    
    # Create summary dataframes with last observations (Original)
    cf_summary_df = cf_df.iloc[-1].to_frame().transpose()
    pl_summary_df = pl_df.iloc[-1].to_frame().transpose()
    bs_summary_df = bs_df.iloc[-1].to_frame().transpose()

    # Calculate Cash flow from operating activities (CFO) and Cash flow from investing activities (CFI) (Original)
    cash_flow_operating_original = cf_summary_df.filter(regex='^CFO').sum(axis=1).iloc[0]
    cash_flow_investing_original = cf_summary_df.filter(regex='^CFI').sum(axis=1).iloc[0]

    # Calculate sales income, cost of sales, and gross profit/loss (Original)
    sales_income_original = pl_summary_df[['Sales - wholesale', 'Service/Fee Income']].sum(axis=1).iloc[0]
    cost_of_sales_original = pl_summary_df.filter(regex='COS$').sum(axis=1).iloc[0]
    gross_profit_loss_original = sales_income_original + cost_of_sales_original
    
    # Calculate total expenses (Original)
    expense_categories = ['Advertising/Promotional', 'Auto', 'Bad debts', 'Bank charges', 'Charitable Contributions',
                          'Cost of Labour', 'Dues and Subscriptions', 'Equipment rental', 'Income tax expense',
                          'Legal and professional fees', 'Office/General Administrative Expenses', 'Other Miscellaneous Service Cost',
                          'Other selling expenses', 'Payroll Expenses', 'Rent or Lease of Buildings', 'Repair and maintenance',
                          'Shipping and delivery expense', 'Travel expenses - selling expense', 'Utilities']

    expenses_original = pl_summary_df[expense_categories].sum(axis=1).iloc[0]
    
    # Calculate other expenses (Original)
    other_expense_categories = ['Depreciation', 'Exchange Gain or Loss', 'Penalties and settlements']
    other_expenses_original = pl_summary_df[other_expense_categories].sum(axis=1).iloc[0]

    # Calculate other income (Original)
    other_income_categories = ['Interest earned', 'Loss on disposal of assets', 'Other Miscellaneous Income', 'Other operating income']
    other_income_original = pl_summary_df[other_income_categories].sum(axis=1).iloc[0]

    # Calculate cash balance from cf_summary_df (Original)
    cash_balance_original = cf_summary_df.sum(axis=1).iloc[0]

    # Calculate net profit/loss from pl_summary_df (Original)
    net_profit_loss_original = gross_profit_loss_original + expenses_original + other_expenses_original + other_income_original
    
    # Calculate total assets (Original)
    ca_columns_original = bs_summary_df.filter(regex='^CA').columns
    nca_columns_original = bs_summary_df.filter(regex='^NCA').columns
    total_assets_original = bs_summary_df[ca_columns_original].sum(axis=1).iloc[0] + bs_summary_df[nca_columns_original].sum(axis=1).iloc[0]
    
    # Calculate total liabilities (Original)
    total_lia_original = -1 * bs_summary_df.filter(regex='^CL').sum(axis=1).iloc[0]
    
    # Calculate total shareholders` equity (Original)
    total_se_original =  bs_summary_df.filter(regex='^SE').sum(axis=1).iloc[0] + net_profit_loss_original

    # Create summary dataframes with last observations (Updated)
    cf_summary_df_updated = cf_df.iloc[-1].to_frame().transpose()
    pl_summary_df_updated = pl_df.iloc[-1].to_frame().transpose()
    bs_summary_df_updated = bs_df.iloc[-1].to_frame().transpose()

    # Adjust values in cf_summary_df based on aggregated_change and cf_inflow
    for index, row in aggregated_change.iterrows():
        variable = row['Variable']
        if variable in cf_summary_df_updated.columns:
            if variable in cf_inflow:
                cf_summary_df_updated[variable] = row['Predicted Value']
            else:
                cf_summary_df_updated[variable] = -1 * row['Predicted Value']

    # Adjust values in pl_summary_df based on aggregated_change and pl_income
    for index, row in aggregated_change.iterrows():
        variable = row['Variable']
        if variable in pl_summary_df_updated.columns:
            if variable in pl_income:
                pl_summary_df_updated[variable] = row['Predicted Value']
            else:
                pl_summary_df_updated[variable] = -1 * row['Predicted Value']
    
    # Adjust values in bs_summary_df based on aggregated_change
    for index, row in aggregated_change.iterrows():
        variable = row['Variable']
        if variable in bs_summary_df_updated.columns:
            if variable in bs_cl:
                bs_summary_df_updated[variable] = row['Predicted Value']
            else:
                bs_summary_df_updated[variable] = -1 * row['Predicted Value']
                
    # Calculate Cash flow from operating activities (CFO) and Cash flow from investing activities (CFI) (Updated)
    cash_flow_operating_updated = cf_summary_df_updated.filter(regex='^CFO').sum(axis=1).iloc[0]
    cash_flow_investing_updated = cf_summary_df_updated.filter(regex='^CFI').sum(axis=1).iloc[0]

    # Calculate sales income, cost of sales, and gross profit/loss (Updated)
    sales_income_updated = pl_summary_df_updated[['Sales - wholesale', 'Service/Fee Income']].sum(axis=1).iloc[0]
    cost_of_sales_updated = pl_summary_df_updated.filter(regex='COS$').sum(axis=1).iloc[0]
    gross_profit_loss_updated = sales_income_updated + cost_of_sales_updated
    
    # Calculate total expenses (Updated)
    expenses_updated = pl_summary_df_updated[expense_categories].sum(axis=1).iloc[0]
    
    # Calculate other expenses (Updated)
    other_expenses_updated = pl_summary_df_updated[other_expense_categories].sum(axis=1).iloc[0]

    # Calculate other income (Updated)
    other_income_updated = pl_summary_df_updated[other_income_categories].sum(axis=1).iloc[0]

    # Calculate cash balance from cf_summary_df (Updated)
    cash_balance_updated = cf_summary_df_updated.sum(axis=1).iloc[0]

    # Calculate net profit/loss from pl_summary_df (Updated)
    net_profit_loss_updated = gross_profit_loss_updated + expenses_updated + other_expenses_updated + other_income_updated
    
    # Update cash and cash equivalents in bs_summary_df (Updated)
    latest_cash_equivalence = bs_summary_df_updated['CA-Cash and cash equivalents'].iloc[0] + cash_balance_updated
    bs_summary_copy = bs_summary_df_updated.copy()
    bs_summary_copy['CA-Cash and cash equivalents'] = latest_cash_equivalence

    # Calculate total assets (Updated)
    ca_columns_updated = bs_summary_copy.filter(regex='^CA').columns
    nca_columns_updated = bs_summary_copy.filter(regex='^NCA').columns
    total_assets_updated = bs_summary_copy[ca_columns_updated].sum(axis=1).iloc[0] + bs_summary_copy[nca_columns_updated].sum(axis=1).iloc[0]
    
    # Calculate total liabilities (Updated)
    total_lia_updated = -1 * bs_summary_df_updated.filter(regex='^CL').sum(axis=1).iloc[0]
    
    # Calculate total shareholders` equity (Updated)
    total_se_updated =  bs_summary_df_updated.filter(regex='^SE').sum(axis=1).iloc[0] + net_profit_loss_updated

    # Create comparison dataframe
    comparison_data = {
        'Metric': ['Sales Income', 'Cost of Sales', 'Gross Profit/Loss', 'Expenses', 'Other Expenses', 'Other Income',
                   'Net Profit/Loss', 'Cash Flow from Investing Activities', 'Cash Flow from Operating Activities', 'Cash Balance',
                   'Total Assets', 'Total Liabilities', 'Total Shareholders Equity'],
        'Original Value': [sales_income_original, cost_of_sales_original, gross_profit_loss_original, expenses_original, other_expenses_original, other_income_original,
                           net_profit_loss_original, cash_flow_investing_original, cash_flow_operating_original, cash_balance_original,
                           total_assets_original, total_lia_original, total_se_original],
        'Updated Value': [sales_income_updated, cost_of_sales_updated, gross_profit_loss_updated, expenses_updated, other_expenses_updated, other_income_updated,
                          net_profit_loss_updated, cash_flow_investing_updated, cash_flow_operating_updated, cash_balance_updated,
                          total_assets_updated, total_lia_updated, total_se_updated]
    }

    comparison_df = pd.DataFrame(comparison_data)

    # Calculate percentage change
    comparison_df['Percentage Change (%)'] = ((comparison_df['Updated Value'] - comparison_df['Original Value']) / comparison_df['Original Value']) * 100

    return comparison_df



In [10]:
def analyze_changes(df1, target_df):
    """
    Analyzes changes in target variables based on predictors using linear regression models.

    Args:
    - df1 (pd.DataFrame): DataFrame containing predictor variables and target variables.
    - target_df (pd.DataFrame): DataFrame specifying target variables and associated predictors.

    Returns:
    - aggregated_changes_df (pd.DataFrame): DataFrame summarizing aggregated changes in variables.
    """

    # Set NumPy options to display whole numbers
    np.set_printoptions(precision=0, suppress=True)
    
    # Set Pandas options to display whole numbers
    pd.set_option('display.float_format', '{:.0f}'.format)
    
    def lr_create_target_predictor_dfs(df1, target_df):
        """
        Creates linear regression models for each target variable based on its predictors.

        Args:
        - df1 (pd.DataFrame): DataFrame containing predictor variables and target variables.
        - target_df (pd.DataFrame): DataFrame specifying target variables and associated predictors.

        Returns:
        - result_dfs (dict): Dictionary containing DataFrames with regression model coefficients and predictions.
        - regression_models (dict): Dictionary containing trained LinearRegression models for each target variable.
        """
        result_dfs = {}
        regression_models = {}

        # Iterate through each row in target_df to train linear regression models
        for idx, row in target_df.iterrows():
            # Extract target variable and predictor(s) for the current row
            target = row['target']
            predictors = row['predictor']

            # Prepare X (predictors) and y (target variable) dataframes from df1
            X = df1[predictors]
            y = df1[target]

            # Initialize a Linear Regression model
            model = LinearRegression()

            # Fit the model using predictors (X) and target variable (y)
            model.fit(X, y)

            # Retrieve coefficients and intercept from the trained model
            coefficients = model.coef_
            intercept = model.intercept_

            # Store the trained model in regression_models dictionary
            regression_models[target] = model

            # Get the last observation (latest values) from df1
            last_observation = df1.iloc[-1]

            # Create a dictionary to store data for final dataframe creation
            data_dict = {'Target': [y.iloc[-1]], 'Y-Intercept': [intercept]}
            
            # Iterate through each predictor column to gather coefficients and last observation values
            for predictor in X.columns:
                # Add coefficient value and predictor's last observation to data_dict
                data_dict[f'Coefficient_{predictor}'] = [coefficients[X.columns.get_loc(predictor)]]
                data_dict[predictor] = [last_observation[predictor]]

            # Create a DataFrame from data_dict
            final_dataframe = pd.DataFrame(data_dict)

            # Repeat final_dataframe creation for each predictor, concatenate them into a single DataFrame
            final_dataframe = pd.concat([final_dataframe] * len(predictors), ignore_index=True)

            # Store the final_dataframe in result_dfs dictionary with target variable as key
            result_dfs[target] = final_dataframe

        return result_dfs, regression_models

    def lr_update_predictor(row, target_col, intercept_col, predictors, predictor_index, target_df, target, target_change_percentage):
        """
        Updates predictors iteratively to achieve the required percentage change in the target variable.

        Args:
        - row (pd.Series): Current row of DataFrame containing predictor and target variables.
        - target_col (str): Name of the target column in the DataFrame.
        - intercept_col (str): Name of the column containing the intercept value in the DataFrame.
        - predictors (list): List of tuples containing coefficient and predictor column names.
        - predictor_index (int): Index indicating the current predictor being updated.
        - target_df (pd.DataFrame): DataFrame specifying target variables and associated predictors.
        - target (str): Current target variable being analyzed.
        - target_change_percentage (float): Percentage change required in the target variable.

        Returns:
        - row (pd.Series): Updated row with predictor values adjusted to achieve the required target change.
        """
        # Extract intercept value from the row using intercept_col
        intercept = row[intercept_col]

        # Initialize current_predicted with the intercept value
        current_predicted = intercept

        # Calculate current_predicted by iterating through predictors
        for coef_col, pred_col in predictors:
            # Update current_predicted using coefficient and predictor values from the row
            current_predicted += row[coef_col] * row[pred_col]

        # Retrieve original_target value from the row using target_col
        original_target = row[target_col]

        # Calculate new_target based on original_target and target_change_percentage
        new_target = original_target * (1 + (target_change_percentage / 100))

        # Extract coef_col and pred_col from predictors at predictor_index
        coef_col, pred_col = predictors[predictor_index]

        # Calculate change_needed based on new_target, current_predicted, and coef_col
        change_needed = (new_target - current_predicted) / row[coef_col]

        # Update pred_col in the row by adding change_needed
        row[pred_col] += change_needed

        # Filter relevant rows in target_df based on pred_col and target, excluding current target
        relevant_rows = target_df[
            target_df['predictor'].apply(lambda preds: pred_col in preds or target in preds) &
            (target_df['target'] != target)
        ]

        # If relevant_rows is not empty, gather relevant information
        if not relevant_rows.empty:
            # Extract unique relevant target values and join them into a string
            relevant_targets = ', '.join(relevant_rows['target'].astype(str).unique())
            row['Relevant_Targets'] = relevant_targets

            # Initialize match_info list to store matching information
            match_info = []

            # Iterate through relevant_rows to gather match information
            for _, rel_row in relevant_rows.iterrows():
                if pred_col in rel_row['predictor']:
                    # Append information about matching pred_col in relevant target
                    match_info.append(f"{rel_row['target']} (matches {pred_col})")
                elif target in rel_row['predictor']:
                    # Append information about matching target in relevant target
                    match_info.append(f"{rel_row['target']} (matches {target})")

            # Join match_info list into a semicolon-separated string and store in row['Match_Info']
            row['Match_Info'] = '; '.join(match_info)

        # Set 'Changed Predictor' in row to pred_col if pred_col exists in row index, otherwise set to target
        row['Changed Predictor'] = pred_col if pred_col in row.index else target


        return row
    
    def calculate_source_impact(ori_df, target, target_df, regression_models, df1):
        """
        Calculates the impact of changes in target variables on their predictors.

        Args:
        - ori_df (pd.DataFrame): DataFrame containing updated predictor values and relevant information.
        - target (str): Target variable being analyzed.
        - target_df (pd.DataFrame): DataFrame specifying target variables and associated predictors.
        - regression_models (dict): Dictionary containing trained LinearRegression models for each target variable.
        - df1 (pd.DataFrame): DataFrame containing predictor variables and target variables.

        Returns:
        - source_impact_df (pd.DataFrame): DataFrame summarizing the impact of changes on predictor variables.
        """
        source_impact_results = []

        def calculate_impact_recursively(current_target, required_value):
            """
            Recursively calculates the impact of a required value change on predictors of the current target variable.

            Args:
            - current_target (str): The current target variable for which impact is being calculated.
            - required_value (float): The required value that needs to be achieved for the current target variable.

            Returns:
            - source_impact_df (DataFrame): DataFrame containing the impact results including required changes,
              original values, and percentage changes for each predictor.

            """
            # Retrieve relevant rows from target_df based on current_target
            relevant_rows = target_df[target_df['target'] == current_target]
            
            if not relevant_rows.empty:
                # Extract predictor information
                predictors_str = relevant_rows['predictor'].values[0]
                predictors = predictors_str.split() if isinstance(predictors_str, str) else predictors_str

                # Retrieve regression model for the current target
                model = regression_models[current_target]

                # Calculate impact for each predictor
                for predictor in predictors:
                    coef = model.coef_[predictors.index(predictor)]
                    intercept = model.intercept_

                    # Calculate the required change in predictor value
                    required_change = (required_value - intercept) / coef

                    # Retrieve original value of the predictor from df1
                    original_value = df1[predictor].iloc[-1]

                    # Calculate percentage change, handling division by zero scenario
                    if original_value != 0:
                        percentage_change = ((required_change - original_value) / original_value) * 100
                    else:
                        percentage_change = "Changed from 0, can't be calculated"

                    # Append the impact results to source_impact_results list
                    source_impact_results.append({
                        'Target Variable': current_target,
                        'Required Value': required_value,
                        'Changed Predictor': predictor,
                        'Required Predictor Value': required_change,
                        'Original Predictor Value': original_value,
                        'Percentage Change (%)': percentage_change
                    })

                    # Recursively call calculate_impact_recursively for the predictor
                    calculate_impact_recursively(predictor, required_change)

            else:
                # Append a default record when no relevant predictors are found
                source_impact_results.append({
                    'Target Variable': current_target,
                    'Required Value': required_value,
                    'Changed Predictor': None,
                    'Required Predictor Value': None,
                    'Original Predictor Value': None,
                    'Percentage Change (%)': None
                })
                
        for index, row in ori_df.iterrows():
            changed_predictor = row['Changed Predictor']
            required_value = row[changed_predictor]
            calculate_impact_recursively(changed_predictor, required_value)

        source_impact_df = pd.DataFrame(source_impact_results)
        return source_impact_df
    
    def find_reverse_impact_hierarchy(target, target_df):
        """
        Finds the hierarchy of reverse impacts starting from the target variable.

        Args:
        - target (str): Target variable for which reverse impact hierarchy needs to be found.
        - target_df (pd.DataFrame): DataFrame specifying target variables and associated predictors.

        Returns:
        - reverse_impact_hierarchy (list): List containing the hierarchy of reverse impacts.
        """
        reverse_impact_hierarchy = []

        def calculate_reverse_impact_hierarchy(current_target):
            # Check if current_target is already in reverse_impact_hierarchy or is the target itself to avoid cycles
            if current_target in reverse_impact_hierarchy or current_target == target:
                return

            # Add current_target to reverse_impact_hierarchy list
            reverse_impact_hierarchy.append(current_target)

            # Retrieve rows from target_df where target equals current_target
            relevant_rows = target_df[target_df['target'] == current_target]

            # Check if relevant_rows is not empty
            if not relevant_rows.empty:
                # Extract predictors_str from relevant_rows and split it into a list if it's a string
                predictors_str = relevant_rows['predictor'].values[0]
                predictors = predictors_str.split() if isinstance(predictors_str, str) else predictors_str

                # Iterate through predictors
                for predictor in predictors:
                    # Recursively call calculate_reverse_impact_hierarchy for each predictor
                    calculate_reverse_impact_hierarchy(predictor)

        # Iterate through predictors associated with the target variable in target_df and call calculate_reverse_impact_hierarchy
        for predictor in target_df[target_df['target'] == target]['predictor'].values[0]:
            calculate_reverse_impact_hierarchy(predictor)
    
        return reverse_impact_hierarchy
    
    def find_impact(stop_variable, stop_value, target, target_df, regression_models, df1):
        """
        Finds combined impacts of changes in predictors on target variables and vice versa.

        Args:
        - stop_variable (str): Variable where to start the impact calculation.
        - stop_value (float): New value for the stop variable.
        - target (str): Predefined target variable to determine upstream or downstream impact.
        - target_df (pd.DataFrame): DataFrame specifying target variables and associated predictors.
        - regression_models (dict): Dictionary containing trained LinearRegression models for each target variable.
        - df1 (pd.DataFrame): DataFrame containing predictor variables and target variables.

        Returns:
        - impact_df (pd.DataFrame): DataFrame summarizing the combined impacts of changes on target variables.
        """
        impact_results = []

        def calculate_impact(current_variable, current_value, source):
            """
            Recursively calculate the impact of a changed variable on its target variables.
            
            Args:
            - current_variable (str): The current variable being analyzed.
            - current_value (float): The new value of the current variable.
            - source (str): Indicates whether the impact is 'Upstream' or 'Downstream'.
            """
            # Update the value of the current variable in the DataFrame
            df1.loc[df1.index[-1], current_variable] = current_value

            # Search for regression models that use the current variable as a predictor
            relevant_rows = target_df[target_df['predictor'].apply(lambda preds: current_variable in preds.split() if isinstance(preds, str) else current_variable in preds)]

            # If no relevant rows, return
            if relevant_rows.empty:
                return

            # Iterate over each relevant row to calculate the impact on target variables
            for _, row in relevant_rows.iterrows():
                impacted_target = row['target']
                predictors_str = row['predictor']
                predictors = predictors_str.split() if isinstance(predictors_str, str) else predictors_str

                # Retrieve the regression model for the impacted target
                model = regression_models[impacted_target]

                # Create a copy of predictors from df1 and update current_variable with current_value
                X = df1[predictors].copy()
                X.loc[X.index[-1], current_variable] = current_value
                X = X.iloc[[-1]]

                # Predict the value using the model
                predicted_value = model.predict(X)[0]

                # Retrieve the original value of the impacted target
                original_value = df1[impacted_target].iloc[-1]

                # Calculate percentage change, handling division by zero if original_value is 0
                percentage_change = ((predicted_value - original_value) / original_value) * 100 if original_value != 0 else "Changed from 0"

                # Determine the new source based on whether the predefined target is being predicted
                new_source = 'Downstream' if source == 'Downstream' else 'Upstream'

                # Append impact results to impact_results list
                impact_results.append({
                    'Variable': impacted_target,
                    'Original Value': original_value,
                    'Predicted Value': predicted_value,
                    'Percentage Change (%)': percentage_change,
                    'Changed Predictor': current_variable,
                    'Source': new_source
                })
                
                # Recursively calculate the impact for the impacted target only if it's different from the current variable
                if impacted_target != current_variable:
                    calculate_impact(impacted_target, predicted_value, new_source)

        # Start the impact calculation with the stop variable and its new value
        calculate_impact(stop_variable, stop_value, 'Start')

        impact_df = pd.DataFrame(impact_results)
        return impact_df
    
    # Display linear regression models for each step in the reverse impact hierarchy
    def display_regression_models_for_hierarchy(reverse_impact_hierarchy, regression_models,target):
        # Display regression model for the predefined target variable if available
        if target in regression_models:
            target_model = regression_models[target]
            target_equation = f"{target} = {target_model.intercept_:.4f}"
            for predictor, coef in zip(target_model.feature_names_in_, target_model.coef_):
                sign = '+' if coef >= 0 else '-'
                target_equation += f" {sign} {abs(coef):.4f}*{predictor}"
            print(f"\nLinear Regression Model of {target}:")
            print(target_equation)
            
        for idx, var in enumerate(reverse_impact_hierarchy):
            if var in regression_models:
                model = regression_models[var]
                equation = f"{var} = {model.intercept_:.4f}"
                for predictor, coef in zip(model.feature_names_in_, model.coef_):
                    sign = '+' if coef >= 0 else '-'
                    equation += f" {sign} {abs(coef):.4f}*{predictor}"
                print(f"\nLinear Regression Model of {var}:")
                print(equation)

    # Print available target variables
    print("Available targets:")
    print(', '.join(target_df['target'].values))

    # Make a copy of the original DataFrame
    df1_used = df1.copy()

    # Get user input for the target variable and percentage change
    target = input("\nEnter the target account: ")
    percentage = float(input("Enter the percentage change: "))

    # Check if the target variable is in the target_df
    if target not in target_df['target'].values:
        print("The target account is not available. Please choose from the available targets.")
    else:
        while True:

            # Create linear regression models for the target variable and its predictors
            result_dfs, lr_models = lr_create_target_predictor_dfs(df1_used, target_df)
            df = result_dfs[target]

            # Update the target variable to achieve the required percentage change
            new_target = df['Target'].iloc[-1] * (1 + percentage / 100)
            df['Target'] = new_target

            # Identify coefficient and predictor columns
            coef_cols = [col for col in df.columns if col.startswith('Coefficient_')]
            predictor_cols = [col[len('Coefficient_'):] for col in coef_cols]
            predictors = list(zip(coef_cols, predictor_cols))

            # Initialize new columns in df
            df['Percentage Change (%)'] = 0.0
            df['Relevant_Targets'] = ''
            df['Changed Predictor'] = ''
            df['Match_Info'] = ''

            # Update predictors iteratively to achieve the target change
            updated_rows = []
            for i in range(len(df)):
                updated_row = lr_update_predictor(df.iloc[i].copy(), 'Target', 'Y-Intercept', predictors, i, target_df, target, percentage)
                updated_rows.append(updated_row)

            # Convert the list of updated rows back to a DataFrame
            df_updated = pd.DataFrame(updated_rows)

            # Calculate the source impacts of the changes
            source_impact_df = calculate_source_impact(df_updated, target, target_df, lr_models, df1_used)

            # Find the reverse impact hierarchy starting from the target variable
            reverse_impact_hierarchy = find_reverse_impact_hierarchy(target, target_df)
            hierarchy_details = []

            # Function to calculate the required predictor value using regression model
            def calculate_required_value(model, target_value):
                intercept = model.intercept_
                coefficients = model.coef_
                required_value = (target_value - intercept) / coefficients[0]  # Assuming a single predictor for simplicity
                return required_value
            
            # Create a list starting with the target and followed by reverse_impact_hierarchy
            variable_list = [target] + reverse_impact_hierarchy
            fitted_value_list = []
            
            # Loop to gather fitted_value_predictor for each variable in variable_list
            for idx, var in enumerate(variable_list):
                if var in lr_models:
                    model = lr_models[var]
                    if idx == 0:
                        target_last_value = df1_used[var].iloc[-1] if var in df1_used.columns else 'N/A'
                        fitted_value_predictor = calculate_required_value(model, target_last_value)
                    else:
                        fitted_value_predictor = calculate_required_value(model, fitted_value_predictor)
                    predictor_variables = []

                    # Find corresponding predictor variable from target_df
                    if var in target_df['target'].values:
                        predictor_variables = target_df.loc[target_df['target'] == var, 'predictor'].iloc[0]
                        if isinstance(predictor_variables, list):
                            predictor_variables = predictor_variables[0]  # Assuming only one predictor for simplicity

                else:
                    fitted_value_predictor = 'N/A'
                    predictor_variables = []

                # Collect fitted_value_predictor into fitted_value_list
                fitted_value_list.append({
                    "Variable": predictor_variables,
                    "Fitted Value Predictor": f"{fitted_value_predictor:.0f}" if isinstance(fitted_value_predictor, (int, float)) else fitted_value_predictor,
                })
            
            # Remove the last element from fitted_value_list
            if fitted_value_list:
                fitted_value_list.pop()
                
                
            # Second loop to collect details into hierarchy_details based on the Variable
            for idx, var_info in enumerate(fitted_value_list):
                var = var_info["Variable"]
                fitted_value_predictor = var_info["Fitted Value Predictor"]
                
                # Ensure fitted_value_predictor is numeric
                try:
                    fitted_value_predictor = float(fitted_value_predictor)
                except ValueError:
                    print(f"Error: Fitted Value Predictor for {var} is not numeric: {fitted_value_predictor}")

                # Determine the required value for the variable
                found_in_target = False
                required_value = None

                # Search source_impact_df for required value based on var
                for index, row in source_impact_df.iterrows():
                    if row['Target Variable'] == var:
                        required_value = row['Required Value']
                        found_in_target = True
                        break

                if not found_in_target:
                    for index, row in source_impact_df.iterrows():
                        if row['Changed Predictor'] == var:
                            required_value = row['Required Predictor Value']
                            break
                
                # Ensure required_value is numeric
                try:
                    required_value = float(required_value)
                except (ValueError, TypeError):
                    print(f"Error: Required Value for {var} is not numeric: {required_value}")
                    continue

                # Calculate the percentage change
                if isinstance(fitted_value_predictor, (int, float)) and required_value is not None:
                    percentage_change = ((required_value - fitted_value_predictor) / fitted_value_predictor) * 100
                    percentage_change_str = f"{percentage_change:.2f}%"
                else:
                    percentage_change_str = "N/A"

                # Append the details to hierarchy_details
                hierarchy_details.append({
                    "Index": idx + 1,
                    "Variable": var,
                    "Fitted Value": f"{fitted_value_predictor:.0f}" if isinstance(fitted_value_predictor, (int, float)) else fitted_value_predictor,
                    "Required Value": f"{required_value:.0f}" if isinstance(required_value, (int, float)) else required_value,
                    "Percentage Change": percentage_change_str
                })

            # Print the hierarchy details in a table format
            print("\nUpstream variables:")
            print(tabulate(hierarchy_details, headers="keys", tablefmt="pretty"))
            
            # Display linear regression models up that used the variables in the reverse impact hierarchy as predictor 
            display_regression_models_for_hierarchy(reverse_impact_hierarchy, lr_models,target)

            # Automatically set stop variable if only one variable in the hierarchy
            if len(reverse_impact_hierarchy) == 1:
                stop_var_index = 0
                print("\nOnly one variable in the hierarchy. Automatically setting changes variable to:", reverse_impact_hierarchy[0])
            else:
                stop_var_index = int(input("\nEnter the number corresponding to the variable where you want to adjust: ")) - 1

            stop_variable = reverse_impact_hierarchy[stop_var_index] if 0 <= stop_var_index < len(reverse_impact_hierarchy) else None

            print("Changed Variable:", stop_variable)
            
            target_of_stop_variable = reverse_impact_hierarchy[stop_var_index] if 0 <= stop_var_index-1 < len(reverse_impact_hierarchy) else None
            
            # Ask user whether to adjust stop variable by required value or percentage
            adjust_by_required_value = input(f"Do you want to adjust '{stop_variable}' by required value? (yes/no): ").strip().lower()

            if adjust_by_required_value == 'yes':
                # Ensure required value corresponds to the stop variable
                stop_var_required_value = None
                for item in hierarchy_details:
                    if item["Variable"] == stop_variable:
                        stop_var_required_value = item["Required Value"]
                        break
                        
                # Adjust stop variable to the required value
                original_stop_value = df1_used[stop_variable].iloc[-1]
                stop_variable_adjustment = (float(stop_var_required_value) - original_stop_value) / original_stop_value * 100
                df1_used.loc[df1_used.index[-1], stop_variable] = float(stop_var_required_value)
                stop_value = float(stop_var_required_value)
                print(stop_value)
                
            else:
                # Obtained the fitted value of stop_variable
                def get_fitted_value_by_variable(hierarchy_details, variable):
                    for item in hierarchy_details:
                        if item["Variable"] == variable:
                            return item["Fitted Value"]
                    return None

                fitted_stop_value = get_fitted_value_by_variable(hierarchy_details, stop_variable)

                if fitted_stop_value is not None:
                    try:
                        fitted_stop_value = float(fitted_stop_value)  # Convert fitted_stop_value to float
                    except ValueError:
                        raise ValueError(f"Fitted value for {stop_variable} is not a valid number: {fitted_stop_value}")
                
                # Adjust stop variable by percentage
                stop_variable_adjustment = float(input(f"Enter the percentage adjustment for '{stop_variable}': "))
                new_stop_value = fitted_stop_value * (1 + stop_variable_adjustment / 100)
                df1_used.loc[df1_used.index[-1], stop_variable] = new_stop_value
                stop_value = new_stop_value

            # Find the combined impacts of the changes
            combined_impact_df = find_impact(stop_variable, stop_value, target, target_df, lr_models, df1_used)

            added_variables = set()
            all_changes = []

            # Collect all unique changes
            for _, row in combined_impact_df.iterrows():
                variable = row['Variable']
                if variable not in added_variables:
                    all_changes.append({'Variable': variable, 'Predicted Value': row['Predicted Value'], 'Change (%)': row['Percentage Change (%)']})
                    added_variables.add(variable)

            if stop_variable not in added_variables:
                all_changes.append({'Variable': stop_variable, 'Predicted Value': stop_value, 'Change (%)': stop_variable_adjustment})
                added_variables.add(stop_variable)

            for i in range(len(df_updated)):
                row = df_updated.iloc[i]
                variable = row['Changed Predictor']
                if variable not in added_variables:
                    all_changes.append({'Variable': variable, 'Predicted Value': row[variable], 'Change (%)': row['Percentage Change (%)']})
                    added_variables.add(variable)

            if target not in added_variables:
                all_changes.append({'Variable': target, 'Predicted Value': new_target, 'Change (%)': percentage})
                added_variables.add(target)

            # Create DataFrame for aggregated changes
            aggregated_changes_df = pd.DataFrame(all_changes).groupby('Variable', as_index=False).sum()

            # Calculate the difference between new_target and the final predicted value of the target
            final_predicted_value = combined_impact_df.loc[combined_impact_df['Variable'] == target, 'Predicted Value'].values[0]
            difference = (final_predicted_value - new_target) / new_target * 100
            print(f"\nBy adjusting the changed variable, the difference between the targeted value ({round(new_target)}) and the final predicted value ({round(final_predicted_value)}) of the {target} is {round(difference)}%.")

            # Calculate the financial metrics
            comparison_df = calculate_metrics(df1, aggregated_changes_df)
            print('\nFinancial Metrics')
            html_table_metrics = comparison_df.to_html(index=False)
            scrollable_div_metrics = f"""
            <div style="overflow-x: auto; white-space: nowrap;">
                {html_table_metrics}
            </div>
            """
            display(HTML(scrollable_div_metrics))
            
            # Display the aggregated changes
            print("\n\nAggregated Changes:")
            html_table_aggregated = aggregated_changes_df.to_html(index=False)
            scrollable_div_aggregated = f"""
            <div style="overflow-x: auto; white-space: nowrap;">
                {html_table_aggregated}
            </div>
            """
            display(HTML(scrollable_div_aggregated))

            # Optionally display source of changes
            show_source = input("\nDo you want to know the source of the changes? (yes/no): ").strip().lower()
            if show_source == 'yes':
                print("\nSource of Changes:")
                html_table_combined = combined_impact_df.to_html(index=False)
                scrollable_div_combined = f"""
                <div style="overflow-x: auto; white-space: nowrap;">
                    {html_table_combined}
                </div>
                """
                display(HTML(scrollable_div_combined))

            # Ask if the user wants to continue the analysis
            choice = input("\nDo you want to continue the analysis? (yes/no): ").strip().lower()
            if choice == 'no':
                print("Analysis complete. Exiting the program.")
                break
            else:
                continue_analysis = input("\nDo you want to continue the analysis based on the prediction previously? (yes/no): ").strip().lower()
                if continue_analysis == 'no':
                    df1_used = df1.copy()
                    same_target = input("\nAre you analyzing the same target? (yes/no): ").strip().lower()
                    if same_target == 'yes':
                        print(f"The target account is  '{target}'")
                        percentage = float(input("Enter the percentage change: "))
                    else:
                        target = input("\nEnter the new target variable: ")
                        percentage = float(input("Enter the percentage change: "))

                        # Check if the target variable is in the target_df
                        if target not in target_df['target'].values:
                            print("The target account is not available. Please choose from the available targets.")
                            continue
                    continue
                else:
                    same_target = input("\nAre you analyzing the same target? (yes/no): ").strip().lower()
                    if same_target == 'yes':
                        print(f"The target account is  '{target}'")
                        percentage = float(input("Enter the percentage change: "))
                    else:
                        target = input("\nEnter the new target variable: ")
                        percentage = float(input("Enter the percentage change: "))

                        # Check if the target variable is in the target_df
                        if target not in target_df['target'].values:
                            print("The target account is not available. Please choose from the available targets.")
                            continue


    print("End of program.")
    return aggregated_changes_df


In [ ]:
aggregated_change = analyze_changes(ma_whole_df, target_df)

Available targets:
CA-Investments - Other_eb, CA-Loans to Others_eb, CA-Other current assets_eb, CA-Prepaid Expenses_eb, CA-Trade and other receivables_eb, CFI-Interest earned, CFI-Other current assets, CFO-Accrued liabilities, CFO-Advertising/Promotional, CFO-Cost of Labour, CFO-Cost of Labour - COS, CFO-Exchange Gain or Loss, CFO-Legal and professional fees, CFO-Office/General Administrative Expenses, CFO-Other Miscellaneous Service Cost, CFO-Other costs of sales - COS, CFO-Other current assets, CFO-Payroll Expenses, CFO-Prepaid Expenses, CFO-Rent or Lease of Buildings, CFO-Repair and maintenance, CFO-Supplies and materials - COS, CFO-Trade and other receivables, CFO-Travel expenses - selling expense, CL-Accrued liabilities_eb, CL-Client Trust Accounts - Liabilities_eb, CL-GST Payable_eb, CL-Other current liabilities_eb, CL-Trade and other payables_eb, Cost of Labour, Cost of Labour - COS, Depreciation, Dues and Subscriptions, Interest earned, NCA-Accumulated depreciation on property